In [2]:
import backtrader as bt
import pandas as pd
import numpy as np
import quandl
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set your Quandl API key
quandl.ApiConfig.api_key = "vJSxz6aLFGy93K1pDtzx"

# Define the two securities you want to trade
security1 = "AAPL"
security2 = "MSFT"

# Retrieve historical price data using Quandl
data1 = quandl.get("WIKI/" + security1, start_date="2018-01-01", end_date="2022-01-01")
data2 = quandl.get("WIKI/" + security2, start_date="2018-01-01", end_date="2022-01-01")



In [3]:
# Calculate the spread between the two securities' prices
spread = data1['Adj. Close'] - data2['Adj. Close']

# Compute z-score of the spread
zscore = (spread - np.mean(spread)) / np.std(spread)

# Create a training dataset for SVM
X = zscore.values[:-1].reshape(-1, 1)
y = np.where(zscore.values[1:] > 0, 1, -1)  # -1 for short signals

In [4]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Initialize SVM classifier
svm_classifier = SVC(kernel='linear')

# Train SVM on the training data
svm_classifier.fit(X_train, y_train)

# Predict trading signals on the testing data
y_pred = svm_classifier.predict(X_test)

In [5]:
# Backtrader Strategy
class SVMStrategy(bt.Strategy):
    params = (
        ("svm_model", None),
    )

    def __init__(self):
        self.svm_model = self.params.svm_model
        self.data_close = self.data0.close
        self.signal = self.data1.signal

    def next(self):
        if self.signal[0] == 1:  # Long signal
            self.buy(size=1)
        elif self.signal[0] == -1:  # Short signal
            self.sell(size=1)

In [6]:
# Create Backtrader data feeds
class QuandlData(bt.feeds.PandasData):
    cols = ('date', 'open', 'high', 'low', 'close', 'volume', 'signal')

In [9]:
print(len(y_pred))

12


In [7]:
# Combine data and predicted signals
combined_data = pd.concat([spread.shift(-1), pd.Series(y_pred, index=spread.index[1:])], axis=1)
combined_data.columns = ['spread', 'signal']

# Create a Backtrader Cerebro engine
cerebro = bt.Cerebro()


ValueError: Length of values (12) does not match length of index (58)

In [ ]:
# Add data feed
data_feed = QuandlData(dataname=combined_data, datetime=combined_data.index, open=None, high=None, low=None, close='spread', volume=None, signal='signal')
cerebro.adddata(data_feed)


In [ ]:
# Add strategy with SVM model
cerebro.addstrategy(SVMStrategy, svm_model=svm_classifier)

# Set initial cash amount
cerebro.broker.set_cash(100000.0)


In [ ]:
# Set commission
cerebro.broker.setcommission(commission=0.001)


In [ ]:

# Print the starting cash
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

In [ ]:

# Run the backtest
cerebro.run()

# Print the ending cash
print("Ending Portfolio Value: %.2f" % cerebro.broker.getvalue())


In [ ]:

# Plot PnL
cerebro.plot(style='candle')

In [10]:
import backtrader as bt
import pandas as pd
import numpy as np
import quandl
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set your Quandl API key
quandl.ApiConfig.api_key = "vJSxz6aLFGy93K1pDtzx"

# Define the two securities you want to trade
security1 = "AAPL"
security2 = "MSFT"

# Retrieve historical price data using Quandl
data1 = quandl.get("WIKI/" + security1, start_date="2018-01-01", end_date="2022-01-01")
data2 = quandl.get("WIKI/" + security2, start_date="2018-01-01", end_date="2022-01-01")

# Calculate the spread between the two securities' prices
spread = data1['Adj. Close'] - data2['Adj. Close']

# Compute z-score of the spread
zscore = (spread - np.mean(spread)) / np.std(spread)

# Create a training dataset for SVM
X = zscore.values[:-1].reshape(-1, 1)
y = np.where(zscore.values[1:] > 0, 1, -1)  # -1 for short signals

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Initialize SVM classifier
svm_classifier = SVC(kernel='linear')

# Train SVM on the training data
svm_classifier.fit(X_train, y_train)

# Predict trading signals on the testing data
y_pred = svm_classifier.predict(X_test)

# Backtrader Strategy
class SVMStrategy(bt.Strategy):
    params = (
        ("svm_model", None),
    )

    def __init__(self):
        self.svm_model = self.params.svm_model
        self.data_close = self.data0.close
        self.signal = self.data1.signal

    def next(self):
        if self.signal[0] == 1:  # Long signal
            self.buy(size=1)
        elif self.signal[0] == -1:  # Short signal
            self.sell(size=1)

# Create Backtrader data feeds
class PandasData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', None),
        ('signal', None),
    )

# Combine data and predicted signals
combined_data = pd.concat([spread.shift(-1), pd.Series(y_pred, index=spread.index[1:])], axis=1)
combined_data.columns = ['spread', 'signal']

# Create a Backtrader Cerebro engine
cerebro = bt.Cerebro()

# Add data feed
data_feed = PandasData(dataname=combined_data)
cerebro.adddata(data_feed)

# Add strategy with SVM model
cerebro.addstrategy(SVMStrategy, svm_model=svm_classifier)

# Set initial cash amount
cerebro.broker.set_cash(100000.0)

# Set commission
cerebro.broker.setcommission(commission=0.001)

# Print the starting cash
print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())

# Run the backtest
cerebro.run()

# Print the ending cash
print("Ending Portfolio Value: %.2f" % cerebro.broker.getvalue())

# Plot PnL
cerebro.plot(style='candle')


ValueError: Length of values (12) does not match length of index (58)